## Analysing used car data

Source of data : https://data.world/data-society/used-cars-data
Howver, DQ team has taken a sample of that data set (50k rows) and dirtied it so taht we can go through the full experience of cleaning the data and working on it.

The data dictionary provided with the data 
* dateCrawled - When this ad was first crawled. All field-values are taken from this date.
* name - Name of the car.
* seller - Whether the seller is private or a dealer.
* offerType - The type of listing
* price - The price on the ad to sell the car.
* abtest - Whether the listing is included in an A/B test.
* vehicleType - The vehicle Type.
* yearOfRegistration - The year in which the car was first registered.
* gearbox - The transmission type.
* powerPS - The power of the car in PS.
* model - The car model name.
* kilometer - How many kilometers the car has driven.
* monthOfRegistration - The month in which the car was first registered.
* fuelType - What type of fuel the car uses.
* brand - The brand of the car.
* notRepairedDamage - If the car has a damage which is not yet repaired.
* dateCreated - The date on which the eBay listing was created.
* nrOfPictures - The number of pictures in the ad.
* postalCode - The postal code for the location of the vehicle.
* lastSeenOnline - When the crawler saw this ad last online.

Objective is to clean and analyse the data.

In [3]:
# Importing the libraries
import pandas as pd
import numpy as np

In [4]:
autos = pd.read_csv('autos.csv', encoding='Latin-1')

In [5]:
print(autos.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

As we can see, some of the values in the data set are null. They are
* vehicleType
* gearbox
* model
* fuelType
* notRepairedDamage

While null values are present, they are less than 20% for any column.

5 columns are integer values and rest are object type. Let us see the first few rows and evaluate further.

In [6]:
print(autos.head())

           dateCrawled                                               name  \
0  2016-03-26 17:47:46                   Peugeot_807_160_NAVTECH_ON_BOARD   
1  2016-04-04 13:38:56         BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik   
2  2016-03-26 18:57:24                         Volkswagen_Golf_1.6_United   
3  2016-03-12 16:58:10  Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...   
4  2016-04-01 14:38:50  Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...   

   seller offerType   price   abtest vehicleType  yearOfRegistration  \
0  privat   Angebot  $5,000  control         bus                2004   
1  privat   Angebot  $8,500  control   limousine                1997   
2  privat   Angebot  $8,990     test   limousine                2009   
3  privat   Angebot  $4,350  control  kleinwagen                2007   
4  privat   Angebot  $1,350     test       kombi                2003   

     gearbox  powerPS   model   odometer  monthOfRegistration fuelType  \
0    manuell      158  andere 

### Clean Columns

We saw the names of the column above and below are some observations.
* names are in camelcase
* a few names can be more descriptve 

Essentially we should clean the names for ease of use and descriptiveness. Let us replace column names to python's preferred snakecase and also make it more descriptive.

In [7]:
# Get list of existing columns
existing_columns = autos.columns
print(existing_columns)

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')


In [8]:
# Function to changes a few column names to make it more descriptive
# and rest to snakecase

def update_column_name(column_name):
    if column_name == "yearOfRegistration":
        column_name = "registration_year"
    elif column_name == "monthOfRegistration":
        column_name = "registration_month"
    elif column_name == "notRepairedDamage":
        column_name = "unrepaired_damage"
    elif column_name == "dateCreated":
        column_name = "ad_created"
    else:
        new_column_name = ""
        for character in column_name:
            if character.islower():
                new_column_name += character
            else:
                new_column_name += "_"+character.lower()
        column_name = new_column_name
    return column_name

In [9]:
# Iterating through all columns to make the change and assigning 
# back to the data frame

new_columns = []
for column in existing_columns:
   new_columns.append(update_column_name(column)) 
autos.columns = new_columns

In [10]:
print(autos.head())

          date_crawled                                               name  \
0  2016-03-26 17:47:46                   Peugeot_807_160_NAVTECH_ON_BOARD   
1  2016-04-04 13:38:56         BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik   
2  2016-03-26 18:57:24                         Volkswagen_Golf_1.6_United   
3  2016-03-12 16:58:10  Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...   
4  2016-04-01 14:38:50  Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...   

   seller offer_type   price   abtest vehicle_type  registration_year  \
0  privat    Angebot  $5,000  control          bus               2004   
1  privat    Angebot  $8,500  control    limousine               1997   
2  privat    Angebot  $8,990     test    limousine               2009   
3  privat    Angebot  $4,350  control   kleinwagen               2007   
4  privat    Angebot  $1,350     test        kombi               2003   

     gearbox  power_p_s   model   odometer  registration_month fuel_type  \
0    manuell        15

As we can see, all the column names have been modified to snakecase and we also updated a few column names to make it more descriptive.

### Initial Exploration and Cleaning

Let us start exploring the data.

In [11]:
autos.describe(include="all")

date_crawled         name  seller offer_type  price abtest  \
count                 50000        50000   50000      50000  50000  50000   
unique                48213        38754       2          2   2357      2   
top     2016-03-21 16:37:21  Ford_Fiesta  privat    Angebot     $0   test   
freq                      3           78   49999      49999   1421  25756   
mean                    NaN          NaN     NaN        NaN    NaN    NaN   
std                     NaN          NaN     NaN        NaN    NaN    NaN   
min                     NaN          NaN     NaN        NaN    NaN    NaN   
25%                     NaN          NaN     NaN        NaN    NaN    NaN   
50%                     NaN          NaN     NaN        NaN    NaN    NaN   
75%                     NaN          NaN     NaN        NaN    NaN    NaN   
max                     NaN          NaN     NaN        NaN    NaN    NaN   

       vehicle_type  registration_year  gearbox     power_p_s  model  \
count         44905       50000.000000    47320  50000.000000  47242   
unique            8                NaN        2           NaN    245   
top       limousine                NaN  manuell           NaN   golf   
freq          12859                NaN    36993           NaN   4024   
mean            NaN        2005.073280      NaN    116.355920    NaN   
std             NaN         105.712813      NaN    209.216627    NaN   
min             NaN        1000.000000      NaN      0.000000    NaN   
25%             NaN        1999.000000      NaN     70.000000    NaN   
50%             NaN        2003.000000      NaN    105.000000    NaN   
75%             NaN        2008.000000      NaN    150.000000    NaN   
max             NaN        9999.000000      NaN  17700.000000    NaN   

         odometer  registration_month fuel_type       brand unrepaired_damage  \
count       50000        50000.000000     45518       50000             40171   
unique         13                 NaN         7          40                 2   
top     150,000km                 NaN    benzin  volkswagen              nein   
freq        32424                 NaN     30107       10687             35232   
mean          NaN            5.723360       NaN         NaN               NaN   
std           NaN            3.711984       NaN         NaN               NaN   
min           NaN            0.000000       NaN         NaN               NaN   
25%           NaN            3.000000       NaN         NaN               NaN   
50%           NaN            6.000000       NaN         NaN               NaN   
75%           NaN            9.000000       NaN         NaN               NaN   
max           NaN           12.000000       NaN         NaN               NaN   

                 ad_created  nr_of_pictures   postal_code            last_seen  
count                 50000         50000.0  50000.000000                50000  
unique                   76             NaN           NaN                39481  
top     2016-04-03 00:00:00             NaN           NaN  2016-04-07 06:17:27  
freq                   1946             NaN           NaN                    8  
mean                    NaN             0.0  50813.627300                  NaN  
std                     NaN             0.0  25779.747957                  NaN  
min                     NaN             0.0   1067.000000                  NaN  
25%                     NaN             0.0  30451.000000                  NaN  
50%                     NaN             0.0  49577.000000                  NaN  
75%                     NaN             0.0  71540.000000                  NaN  
max                     NaN             0.0  99998.000000                  NaN

From here itself, there are a few observations and following to-dos which can be made.

* Name of the car which is offered most is Ford Fiesta.
* There is just 2 sellers and out of the two, almost all cars are offered  by just 1 seller - 49,999. Let us evaluate this further to see what is the other value. We may remove this column for our analysis as all the values are same and it would not provide enough input or differentiation.
* Just like seller, offer_type also has just 2 unique values and almost all the offers are of type "Angebot". We will also check this further.
* price, while a numeric value, is stored as a string. Let us convert that to numeric.
* abtest has 2 values which makes sense as it would denote whether listing is being ab tested or not.
* We have 8 unique vechicle types with limousine being the top one with a frequency of 12859.
* odometer, (just like price) while a numeric value, is stored as a string. Let us convert that to numeric.
* There are 40 different brands with volkswagen being at the top as it has been mentioned 40 times.
* We can ignore nr_of_pictures as it is 0 for all rows.

We defined a few to-dos here so lets go through those columns in detail.

In [12]:
seller = autos["seller"].value_counts()
seller

privat        49999
gewerblich        1
Name: seller, dtype: int64

In [13]:
offer_type = autos["offer_type"].value_counts()
offer_type

Angebot    49999
Gesuch         1
Name: offer_type, dtype: int64

In [14]:
nr_of_pictures = autos["nr_of_pictures"].value_counts()
nr_of_pictures

0    50000
Name: nr_of_pictures, dtype: int64

As we saw from our earlier analysis, we ccan safely drop these columns as they are all same (almost).

Let us now convert the numeric values stored as text to numeric columns. Let us start with ```price```

In [15]:
prices = autos["price"]
prices.value_counts()

$0          1421
$500         781
$1,500       734
$2,500       643
$1,000       639
            ... 
$356           1
$14,489        1
$310           1
$2,410         1
$198,000       1
Name: price, Length: 2357, dtype: int64

In [16]:
prices.describe()

count     50000
unique     2357
top          $0
freq       1421
Name: price, dtype: object

As we can see, the value stored has a pattern in it wherein it starts with $ and is comma separated. Let us write a function

In [17]:
prices = prices.str.replace("$", "").str.replace(",","")
prices

0         5000
1         8500
2         8990
3         4350
4         1350
         ...  
49995    24900
49996     1980
49997    13200
49998    22900
49999     1250
Name: price, Length: 50000, dtype: object

In [18]:
autos["price"] = prices
autos

date_crawled                                               name  \
0      2016-03-26 17:47:46                   Peugeot_807_160_NAVTECH_ON_BOARD   
1      2016-04-04 13:38:56         BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik   
2      2016-03-26 18:57:24                         Volkswagen_Golf_1.6_United   
3      2016-03-12 16:58:10  Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...   
4      2016-04-01 14:38:50  Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...   
...                    ...                                                ...   
49995  2016-03-27 14:38:19   Audi_Q5_3.0_TDI_qu._S_tr.__Navi__Panorama__Xenon   
49996  2016-03-28 10:50:25  Opel_Astra_F_Cabrio_Bertone_Edition___TÜV_neu+...   
49997  2016-04-02 14:44:48                     Fiat_500_C_1.2_Dualogic_Lounge   
49998  2016-03-08 19:25:42                 Audi_A3_2.0_TDI_Sportback_Ambition   
49999  2016-03-14 00:42:12                                Opel_Vectra_1.6_16V   

       seller offer_type  price   abtest vehicle_type  registration_year  \
0      privat    Angebot   5000  control          bus               2004   
1      privat    Angebot   8500  control    limousine               1997   
2      privat    Angebot   8990     test    limousine               2009   
3      privat    Angebot   4350  control   kleinwagen               2007   
4      privat    Angebot   1350     test        kombi               2003   
...       ...        ...    ...      ...          ...                ...   
49995  privat    Angebot  24900  control    limousine               2011   
49996  privat    Angebot   1980  control       cabrio               1996   
49997  privat    Angebot  13200     test       cabrio               2014   
49998  privat    Angebot  22900  control        kombi               2013   
49999  privat    Angebot   1250  control    limousine               1996   

         gearbox  power_p_s   model   odometer  registration_month fuel_type  \
0        manuell        158  andere  150,000km                   3       lpg   
1      automatik        286     7er  150,000km                   6    benzin   
2        manuell        102    golf   70,000km                   7    benzin   
3      automatik         71  fortwo   70,000km                   6    benzin   
4        manuell          0   focus  150,000km                   7    benzin   
...          ...        ...     ...        ...                 ...       ...   
49995  automatik        239      q5  100,000km                   1    diesel   
49996    manuell         75   astra  150,000km                   5    benzin   
49997  automatik         69     500    5,000km                  11    benzin   
49998    manuell        150      a3   40,000km                  11    diesel   
49999    manuell        101  vectra  150,000km                   1    benzin   

            brand unrepaired_damage           ad_created  nr_of_pictures  \
0         peugeot              nein  2016-03-26 00:00:00               0   
1             bmw              nein  2016-04-04 00:00:00               0   
2      volkswagen              nein  2016-03-26 00:00:00               0   
3           smart              nein  2016-03-12 00:00:00               0   
4            ford              nein  2016-04-01 00:00:00               0   
...           ...               ...                  ...             ...   
49995        audi              nein  2016-03-27 00:00:00               0   
49996        opel              nein  2016-03-28 00:00:00               0   
49997        fiat              nein  2016-04-02 00:00:00               0   
49998        audi              nein  2016-03-08 00:00:00               0   
49999        opel              nein  2016-03-13 00:00:00               0   

       postal_code            last_seen  
0            79588  2016-04-06 06:45:54  
1            71034  2016-04-06 14:45:08  
2            35394  2016-04-06 20:15:37  
3            33729  2016-03-15 03:16:28  
4            39218  2016-04-01 14:38:50  
..

While we have updated our values, it is till text type.

In [19]:
autos["price"].dtype

dtype('O')

In [20]:
autos["price"] = autos["price"].astype(float)
print(autos["price"].dtype)
autos["price"].describe()

float64


count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

Perfect! Now let us do the same for odometer. Let us also change the column name to ```odometer_km```

In [21]:
autos["odometer"].value_counts()

150,000km    32424
125,000km     5170
100,000km     2169
90,000km      1757
80,000km      1436
70,000km      1230
60,000km      1164
50,000km      1027
5,000km        967
40,000km       819
30,000km       789
20,000km       784
10,000km       264
Name: odometer, dtype: int64

In [22]:
autos["odometer"].describe()

count         50000
unique           13
top       150,000km
freq          32424
Name: odometer, dtype: object

As we can see the values are in text in the form of comma separated value with a ```km``` suffix at the end. let us remove that.

In [23]:
autos["odometer"] = autos["odometer"].str.replace(",","").str.replace("km","")
autos.rename({"odometer": "odometer_km"}, axis=1, inplace=True)
autos["odometer_km"] = autos["odometer_km"].astype(float)

We replace the odometer values, changed the name of the column and converted the column to int dtype. Let us check if it worked.

In [24]:
autos.head()
autos["odometer_km"].dtype
autos["odometer_km"].describe()

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

Great. Just converting these to numeric type, give us some good insights.

* Mean ododmeter is upwards of 125k kilometers - ie, the cars have been driven a lot.
* Mean price is less than 10000 USD which makes sense as the cars have a lot of kilometers on them.

However before coming to any conclusions let us continue with the exploration.

Before going any forward let us also drop the columns we found earlier which do not contribute to our analysis.

In [25]:
autos = autos.drop(["nr_of_pictures", "seller", "offer_type"], axis=1)

In [26]:
autos

date_crawled                                               name  \
0      2016-03-26 17:47:46                   Peugeot_807_160_NAVTECH_ON_BOARD   
1      2016-04-04 13:38:56         BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik   
2      2016-03-26 18:57:24                         Volkswagen_Golf_1.6_United   
3      2016-03-12 16:58:10  Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...   
4      2016-04-01 14:38:50  Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...   
...                    ...                                                ...   
49995  2016-03-27 14:38:19   Audi_Q5_3.0_TDI_qu._S_tr.__Navi__Panorama__Xenon   
49996  2016-03-28 10:50:25  Opel_Astra_F_Cabrio_Bertone_Edition___TÜV_neu+...   
49997  2016-04-02 14:44:48                     Fiat_500_C_1.2_Dualogic_Lounge   
49998  2016-03-08 19:25:42                 Audi_A3_2.0_TDI_Sportback_Ambition   
49999  2016-03-14 00:42:12                                Opel_Vectra_1.6_16V   

         price   abtest vehicle_type  registration_year    gearbox  power_p_s  \
0       5000.0  control          bus               2004    manuell        158   
1       8500.0  control    limousine               1997  automatik        286   
2       8990.0     test    limousine               2009    manuell        102   
3       4350.0  control   kleinwagen               2007  automatik         71   
4       1350.0     test        kombi               2003    manuell          0   
...        ...      ...          ...                ...        ...        ...   
49995  24900.0  control    limousine               2011  automatik        239   
49996   1980.0  control       cabrio               1996    manuell         75   
49997  13200.0     test       cabrio               2014  automatik         69   
49998  22900.0  control        kombi               2013    manuell        150   
49999   1250.0  control    limousine               1996    manuell        101   

        model  odometer_km  registration_month fuel_type       brand  \
0      andere     150000.0                   3       lpg     peugeot   
1         7er     150000.0                   6    benzin         bmw   
2        golf      70000.0                   7    benzin  volkswagen   
3      fortwo      70000.0                   6    benzin       smart   
4       focus     150000.0                   7    benzin        ford   
...       ...          ...                 ...       ...         ...   
49995      q5     100000.0                   1    diesel        audi   
49996   astra     150000.0                   5    benzin        opel   
49997     500       5000.0                  11    benzin        fiat   
49998      a3      40000.0                  11    diesel        audi   
49999  vectra     150000.0                   1    benzin        opel   

      unrepaired_damage           ad_created  postal_code            last_seen  
0                  nein  2016-03-26 00:00:00        79588  2016-04-06 06:45:54  
1                  nein  2016-04-04 00:00:00        71034  2016-04-06 14:45:08  
2                  nein  2016-03-26 00:00:00        35394  2016-04-06 20:15:37  
3                  nein  2016-03-12 00:00:00        33729  2016-03-15 03:16:28  
4                  nein  2016-04-01 00:00:00        39218  2016-04-01 14:38:50  
...                 ...                  ...          ...                  ...  
49995              nein  2016-03-27 00:00:00        82131  2016-04-01 13:47:40  
49996              nein  2016-03-28 00:00:00        44807  2016-04-02 14:18:02  
49997              nein  2016-04-02 00:00:00        73430  2016-04-04 11:47:27  
49998              nein  2016-03-08 00:00:00        35683  2016-04-05 16:45:07  
49999              nein  2016-03-13 00:00:00        45897  2016-04-06 21:18:48  

[50000 rows x 17 columns]

Let us continue to explore odometer and price columns

### Exploring Odometer and Price

Let us first analyse the data and see if there are any outliers or not.

In [34]:
prices = autos["price"]
print("Unique prices: \n", prices.unique().shape)
print("Description: \n", prices.describe())
print("First 20 values :\n",prices.value_counts().sort_index().head(20))
print("Sorted prices for 20 values: \n",prices.value_counts().sort_index(ascending=False).head(20))

Unique prices: 
 (2357,)
Description: 
 count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64
First 20 values :
 0.0     1421
1.0      156
2.0        3
3.0        1
5.0        2
8.0        1
9.0        1
10.0       7
11.0       2
12.0       3
13.0       2
14.0       1
15.0       2
17.0       3
18.0       1
20.0       4
25.0       5
29.0       1
30.0       7
35.0       1
Name: price, dtype: int64
Sorted prices for 20 values: 
 99999999.0    1
27322222.0    1
12345678.0    3
11111111.0    2
10000000.0    1
3890000.0     1
1300000.0     1
1234566.0     1
999999.0      2
999990.0      1
350000.0      1
345000.0      1
299000.0      1
295000.0      1
265000.0      1
259000.0      1
250000.0      1
220000.0      1
198000.0      1
197000.0      1
Name: price, dtype: int64


As we see there are 2357 unique prices. Also there are 1421 cars with a price of 0. The highest prices also seems to high - 10 million, 2.8 million and so on. 

We can remove the 1421 rows with 0 price. 1421 / 50000 - 2.8% of the total data set so it should be ok to remove.

But what about the high prices ? What should be kept ? What is an outlier here ? The data is from E-Bay which is an auction site so people may post the auction bid to start at 1 so lets keep that. However at the higher end we see the incremental increase till 350000 and then a sudden jump to 9999990 and above. Let our range be from 1 to 350000.

In [37]:
autos = autos[autos["price"].between(1,350000)]

In [38]:
autos.shape

(48565, 17)

As we see we have removed rows containing 0 price and absurdly high prices. Let us evalute ```odometer_km``` now.

In [39]:
odometer_kms = autos["odometer_km"]

In [41]:
print("Unique readings: \n", odometer_kms.unique().shape)
print("Description: \n", odometer_kms.describe())
print("First 20 values :\n",odometer_kms.value_counts().sort_index().head(20))
print("Sorted odo readings desc for 20 values: \n",odometer_kms.value_counts().sort_index(ascending=False).head(20))

Unique readings: 
 (13,)
Description: 
 count     48565.000000
mean     125770.101925
std       39788.636804
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64
First 20 values :
 5000.0        836
10000.0       253
20000.0       762
30000.0       780
40000.0       815
50000.0      1012
60000.0      1155
70000.0      1217
80000.0      1415
90000.0      1734
100000.0     2115
125000.0     5057
150000.0    31414
Name: odometer_km, dtype: int64
Sorted odo readings desc for 20 values: 
 150000.0    31414
125000.0     5057
100000.0     2115
90000.0      1734
80000.0      1415
70000.0      1217
60000.0      1155
50000.0      1012
40000.0       815
30000.0       780
20000.0       762
10000.0       253
5000.0        836
Name: odometer_km, dtype: int64


The first interesting observation is that for more than 45,000 rows the unique values for odometer readings are just 13. This indicates that those values would have been chosen from some auto select or drop down on the submission page.

The min value is 5,000 km and max is 150,000 km and there are more cars with higher readings than lesser. Interestingly, 150,000 seems to be the most common value as well with greater than 31,000 cars reporting that reading. The values here seem within range and does not necessitate any removal. Let us move to other columns.

### Exploring the dates

From the data dictionary, we see there are 5 columns having date data.

* date_crawled - When this ad was first crawled. All field-values are taken from this date.
* registration_year - The year in which the car was first registered.
* registration_month - The month in which the car was first registered.
* ad_created - The date on which the eBay listing was created.
* last_seen - When the crawler saw this ad last online.

In [46]:
print(autos.info())
autos.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48565 entries, 0 to 49999
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   date_crawled        48565 non-null  object 
 1   name                48565 non-null  object 
 2   price               48565 non-null  float64
 3   abtest              48565 non-null  object 
 4   vehicle_type        43979 non-null  object 
 5   registration_year   48565 non-null  int64  
 6   gearbox             46222 non-null  object 
 7   power_p_s           48565 non-null  int64  
 8   model               46107 non-null  object 
 9   odometer_km         48565 non-null  float64
 10  registration_month  48565 non-null  int64  
 11  fuel_type           44535 non-null  object 
 12  brand               48565 non-null  object 
 13  unrepaired_damage   39464 non-null  object 
 14  ad_created          48565 non-null  object 
 15  postal_code         48565 non-null  int64  
 16  last

price  registration_year     power_p_s    odometer_km  \
count   48565.000000       48565.000000  48565.000000   48565.000000   
mean     5888.935591        2004.755421    117.197158  125770.101925   
std      9059.854754          88.643887    200.649618   39788.636804   
min         1.000000        1000.000000      0.000000    5000.000000   
25%      1200.000000        1999.000000     71.000000  125000.000000   
50%      3000.000000        2004.000000    107.000000  150000.000000   
75%      7490.000000        2008.000000    150.000000  150000.000000   
max    350000.000000        9999.000000  17700.000000  150000.000000   

       registration_month   postal_code  
count        48565.000000  48565.000000  
mean             5.782251  50975.745207  
std              3.685595  25746.968398  
min              0.000000   1067.000000  
25%              3.000000  30657.000000  
50%              6.000000  49716.000000  
75%              9.000000  71665.000000  
max             12.000000  99998.000000

We see that ```registration_year``` and ```registration_month``` are numeric columns and accorindgly we are getting values like mean etc. However, ```date_crawled```, ```ad_created``` and ```last_seen``` are string. Let us convert them to numerical columns so that we will be able to analyse them without any extra data processing.

Let us explore the values in these 3 columns.

In [47]:
autos[["date_crawled", "ad_created", "last_seen"]][0:5]

date_crawled           ad_created            last_seen
0  2016-03-26 17:47:46  2016-03-26 00:00:00  2016-04-06 06:45:54
1  2016-04-04 13:38:56  2016-04-04 00:00:00  2016-04-06 14:45:08
2  2016-03-26 18:57:24  2016-03-26 00:00:00  2016-04-06 20:15:37
3  2016-03-12 16:58:10  2016-03-12 00:00:00  2016-03-15 03:16:28
4  2016-04-01 14:38:50  2016-04-01 00:00:00  2016-04-01 14:38:50

After seeing the values, we see that these are stored as time stamps. Can we use date time here ? - lets come back to this later. For now, we see a patern that the first 10 characters give us the date.

In [55]:
autos["date_crawled"].str[:10]

0        2016-03-26
1        2016-04-04
2        2016-03-26
3        2016-03-12
4        2016-04-01
            ...    
49995    2016-03-27
49996    2016-03-28
49997    2016-04-02
49998    2016-03-08
49999    2016-03-14
Name: date_crawled, Length: 48565, dtype: object

In [67]:
# using the above to calcualte distributions for these columns
date_crawled = autos["date_crawled"].str[:10]
date_crawled.value_counts(normalize=True, dropna=False).sort_index(ascending=False)

2016-04-07    0.001400
2016-04-06    0.003171
2016-04-05    0.013096
2016-04-04    0.036487
2016-04-03    0.038608
2016-04-02    0.035478
2016-04-01    0.033687
2016-03-31    0.031834
2016-03-30    0.033687
2016-03-29    0.034099
2016-03-28    0.034860
2016-03-27    0.031092
2016-03-26    0.032204
2016-03-25    0.031607
2016-03-24    0.029342
2016-03-23    0.032225
2016-03-22    0.032987
2016-03-21    0.037373
2016-03-20    0.037887
2016-03-19    0.034778
2016-03-18    0.012911
2016-03-17    0.031628
2016-03-16    0.029610
2016-03-15    0.034284
2016-03-14    0.036549
2016-03-13    0.015670
2016-03-12    0.036920
2016-03-11    0.032575
2016-03-10    0.032184
2016-03-09    0.033090
2016-03-08    0.033296
2016-03-07    0.036014
2016-03-06    0.014043
2016-03-05    0.025327
Name: date_crawled, dtype: float64

In [69]:
# using the above to calcualte distributions for these columns
last_seen = autos["last_seen"].str[:10]
last_seen.value_counts(normalize=True, dropna=False).sort_index(ascending=False)

2016-04-07    0.131947
2016-04-06    0.221806
2016-04-05    0.124761
2016-04-04    0.024483
2016-04-03    0.025203
2016-04-02    0.024915
2016-04-01    0.022794
2016-03-31    0.023783
2016-03-30    0.024771
2016-03-29    0.022341
2016-03-28    0.020859
2016-03-27    0.015649
2016-03-26    0.016802
2016-03-25    0.019211
2016-03-24    0.019767
2016-03-23    0.018532
2016-03-22    0.021373
2016-03-21    0.020632
2016-03-20    0.020653
2016-03-19    0.015834
2016-03-18    0.007351
2016-03-17    0.028086
2016-03-16    0.016452
2016-03-15    0.015876
2016-03-14    0.012602
2016-03-13    0.008895
2016-03-12    0.023783
2016-03-11    0.012375
2016-03-10    0.010666
2016-03-09    0.009595
2016-03-08    0.007413
2016-03-07    0.005395
2016-03-06    0.004324
2016-03-05    0.001071
Name: last_seen, dtype: float64

In [70]:
# using the above to calcualte distributions for these columns
ad_created = autos["ad_created"].str[:10]
ad_created.value_counts(normalize=True, dropna=False).sort_index(ascending=False)

2016-04-07    0.001256
2016-04-06    0.003253
2016-04-05    0.011819
2016-04-04    0.036858
2016-04-03    0.038855
                ...   
2015-12-05    0.000021
2015-11-10    0.000021
2015-09-09    0.000021
2015-08-10    0.000021
2015-06-11    0.000021
Name: ad_created, Length: 76, dtype: float64

In [74]:
ad_created.unique().shape

(76,)

In [75]:
last_seen.unique().shape

(34,)

In [76]:
date_crawled.unique().shape

(34,)

The last 3 day see almost 50% of the values in last_seen. This would indicate crawler high activity as it is ending rather than last seen ad aka sales as it would be too high sales for just alst 3 days.

All adds have been created during a 10-11 month period.

In [77]:
autos["registration_year"].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

Registration year would help us determine the age of the car. Min and max do not make sense as they are before the car was invented and a future data.

In [78]:
autos["registration_year"].value_counts().head(20)

2000    3156
2005    2936
1999    2897
2004    2703
2003    2699
2006    2670
2001    2636
2002    2486
1998    2363
2007    2277
2008    2215
2009    2085
1997    1951
2011    1623
2010    1589
2017    1392
1996    1373
2012    1310
1995    1227
2016    1220
Name: registration_year, dtype: int64

In [82]:
print(autos["registration_year"].unique().shape)
autos["registration_year"].value_counts().sort_index().head(20)

(95,)


1000    1
1001    1
1111    1
1800    2
1910    5
1927    1
1929    1
1931    1
1934    2
1937    4
1938    1
1939    1
1941    2
1943    1
1948    1
1950    3
1951    2
1952    1
1953    1
1954    2
Name: registration_year, dtype: int64

In [81]:
autos["registration_year"].value_counts().sort_index(ascending=False).head(20)

9999       3
9000       1
8888       1
6200       1
5911       1
5000       4
4800       1
4500       1
4100       1
2800       1
2019       2
2018     470
2017    1392
2016    1220
2015     392
2014     663
2013     803
2012    1310
2011    1623
2010    1589
Name: registration_year, dtype: int64

The minimum value is 1000, before cars were invented
The maximum value is 9999, many years into the future
Also a car cannot be registered after a listing has been seen. So any value with registration year greater than 2016 is not accurate. What about the earliest valid year ? Well the first car was invented in late 1700s so maybe from 1800s can still be considered as there are 2 values from 1800. So maybe we can have a range from 1800 to 2016 inclusive.


In [83]:
autos[autos["registration_year"].between(1900,2016)].shape

(46681, 17)

In [84]:
autos[autos["registration_year"].between(1800,2016)].shape

(46683, 17)

So we can take 1800 to 2016 - should be fine as most of the values fall within that range.

In [85]:
autos = autos[autos["registration_year"].between(1800,2016)]

In [86]:
autos["registration_year"].describe()

count    46683.000000
mean      2002.902063
std          7.306668
min       1800.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       2016.000000
Name: registration_year, dtype: float64

#### Exploring Price by Brand

In [87]:
autos["brand"].describe()

count          46683
unique            40
top       volkswagen
freq            9862
Name: brand, dtype: object

In [94]:
autos["brand"].value_counts(normalize=True).sort_values(ascending=False)

volkswagen        0.211255
bmw               0.110040
opel              0.107577
mercedes_benz     0.096459
audi              0.086563
ford              0.069897
renault           0.047148
peugeot           0.029840
fiat              0.025641
seat              0.018272
skoda             0.016409
nissan            0.015273
mazda             0.015188
smart             0.014159
citroen           0.014009
toyota            0.012703
hyundai           0.010025
sonstige_autos    0.009811
volvo             0.009147
mini              0.008761
mitsubishi        0.008269
honda             0.007840
kia               0.007069
alfa_romeo        0.006641
porsche           0.006126
suzuki            0.005934
chevrolet         0.005698
chrysler          0.003513
dacia             0.002635
daihatsu          0.002506
jeep              0.002271
subaru            0.002142
land_rover        0.002099
saab              0.001649
jaguar            0.001564
daewoo            0.001499
trabant           0.001392
r

There are 40 brands and > 65% of the total is dominated by 6 brands. Let us analyse these 7 brands - they also are more than 5% of the distribution. German manufacturers represent four out of the top five brands, almost 50% of the overall listings. Volkswagen is by far the most popular brand, with approximately double the cars for sale of the next two brands combined.

In [107]:
brands = autos["brand"].value_counts(normalize=True).sort_values(ascending=False).head(6).index
brands

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford'], dtype='object')

In [116]:
brands_price = {}

for brand in brands:
    selected_brands = autos[autos["brand"]==brand]
    mean_price = selected_brands["price"].mean()
    brands_price[brand] = mean_price

In [117]:
brands_price

{'volkswagen': 5402.410261610221,
 'bmw': 8332.820517811953,
 'opel': 2975.2419354838707,
 'mercedes_benz': 8628.450366422385,
 'audi': 9336.687453600594,
 'ford': 3749.4695065890287}

* Ford and Opel less expensive
* Audi, BMW and Mercedes Benz more expensive
* Volkswagen in middle - maybe thats why it has such a high distribution 

#### Exploring mileage by top 6 brands
For the top 6 brands, let's use aggregation to understand the average mileage for those cars and if there's any visible link with mean price. Actually we are exploring odometer reading so calling it mileage is not entirely correct.


In [121]:
brands_odo = {}
for brand in brands:
    selected_brands = autos[autos["brand"]==brand]
    mean_mileage = selected_brands["odometer_km"].mean()
    brands_odo[brand] = mean_mileage
brands_odo

{'volkswagen': 128707.15879132022,
 'bmw': 132572.51313996495,
 'opel': 129310.0358422939,
 'mercedes_benz': 130788.36331334666,
 'audi': 129157.38678544914,
 'ford': 124266.01287159056}

In [122]:
# converting both dict to Series
brands_price_series = pd.Series(brands_price)
brands_odo_series = pd.Series(brands_odo)

In [123]:
brands_price_odo = pd.DataFrame(brands_price_series, columns=["price"])

In [124]:
brands_price_odo["odo_km"] = brands_odo_series

In [125]:
brands_price_odo

price         odo_km
volkswagen     5402.410262  128707.158791
bmw            8332.820518  132572.513140
opel           2975.241935  129310.035842
mercedes_benz  8628.450366  130788.363313
audi           9336.687454  129157.386785
ford           3749.469507  124266.012872